In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from tqdm import tqdm
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

# Convenience for making pretty plots
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
from gpax.kernels import RBFKernel
from gpax.gp import ExactGP
from gpax.state import set_rng_key, silent_mode
from gpax.utils.experiments import Simple2d

# Bayesian Optimization with UCB

We can do some slightly more complicated Bayesian optimization than what was presented in the previous notebook, but let's begin with the simplest once more: using UCB for pure exploration. We'll be using a more complicated function this time, `Simple2d`, which has a few local minima/maxima and one global maxima.

In [ ]:
from gpax.acquisition import UpperConfidenceBound

In [ ]:
set_rng_key(1)
experiment = Simple2d()
ppd = 40
x_grid = experiment.get_dense_coordinates(ppd=ppd)
y_grid = experiment(x_grid)
extent = experiment.get_domain_mpl_extent()

In [ ]:
acqf = UpperConfidenceBound(beta=np.inf, q=1, bounds=experiment.domain)

In [ ]:
# Reset
x = None
y = None
gps = []

In [ ]:
plot_kwargs = {"extent": extent, "interpolation": "nearest", "origin": "lower", "cmap": "viridis"}

with silent_mode():
    for iteration in tqdm(range(20), disable=True):
        gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=500, observation_noise=False)
        if x is not None:
            gp.fit()
        x_star, y_star = acqf.optimize(gp, n=1000, method="Halton")
        y_exp = experiment(x_star)
    
        mu, sd = gp.predict(x_grid, fast=True)
        mu = mu.reshape(ppd, ppd).T
        sd = sd.reshape(ppd, ppd).T
        
        fig, axs = plt.subplots(1, 2, figsize=(6, 2), sharex=True)
        ax = axs[0]
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(mu, **plot_kwargs)
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Experiment")
    
        ax = axs[1]
        v = acqf(gp, x_grid)
        v = v.reshape(ppd, ppd).T
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(v, **plot_kwargs)
        # ax.plot(x_grid, v, "b-")
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Acquisition Function")
        
        plt.show()
    
        if x is not None:
            x = np.append(x, x_star, axis=0)
            y = np.append(y, y_exp)
        else:
            x = np.atleast_2d(x_star).copy()
            y = np.atleast_1d(y_exp).copy()


# Bayesian Optimization with UCB and a finite $\beta$ value

Now, let's try the same thing but using a finite value for $beta$. This will actually attempt to find the maximum of the function instead of just mapping out the space.

In [ ]:
from gpax.acquisition import UpperConfidenceBound

In [ ]:
set_rng_key(1)
experiment = Simple2d()
ppd = 40
x_grid = experiment.get_dense_coordinates(ppd=ppd)
y_grid = experiment(x_grid)
extent = experiment.get_domain_mpl_extent()

In [ ]:
acqf = UpperConfidenceBound(beta=20.0, q=1, bounds=experiment.domain)

In [ ]:
# Reset
x = None
y = None
gps = []

In [ ]:
plot_kwargs = {"extent": extent, "interpolation": "nearest", "origin": "lower", "cmap": "viridis"}

with silent_mode():
    for iteration in tqdm(range(20), disable=True):
        gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=500, observation_noise=False)
        if x is not None:
            gp.fit()
        x_star, y_star = acqf.optimize(gp, n=1000, method="Halton")
        y_exp = experiment(x_star)
    
        mu, sd = gp.predict(x_grid, fast=True)
        mu = mu.reshape(ppd, ppd).T
        sd = sd.reshape(ppd, ppd).T
        
        fig, axs = plt.subplots(1, 2, figsize=(6, 2), sharex=True)
        ax = axs[0]
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(mu, **plot_kwargs)
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Experiment")
    
        ax = axs[1]
        v = acqf(gp, x_grid)
        v = v.reshape(ppd, ppd).T
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(v, **plot_kwargs)
        # ax.plot(x_grid, v, "b-")
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Acquisition Function")
        
        plt.show()
    
        if x is not None:
            x = np.append(x, x_star, axis=0)
            y = np.append(y, y_exp)
        else:
            x = np.atleast_2d(x_star).copy()
            y = np.atleast_1d(y_exp).copy()


# Expected Improvement

Similarly, we can use the `ExpectedImprovement` acquisition function, which is another way of trying to find the maximum.

In [ ]:
from gpax.acquisition import ExpectedImprovement

In [ ]:
set_rng_key(50)
experiment = Simple2d()
ppd = 40
x_grid = experiment.get_dense_coordinates(ppd=ppd)
y_grid = experiment(x_grid)
extent = experiment.get_domain_mpl_extent()

In [ ]:
acqf = ExpectedImprovement(q=1, bounds=experiment.domain, fast=True)

In [ ]:
# Reset
x = None
y = None
gps = []

In [ ]:
plot_kwargs = {"extent": extent, "interpolation": "nearest", "origin": "lower", "cmap": "viridis"}

with silent_mode():
    for iteration in tqdm(range(40), disable=True):
        gp = ExactGP(kernel=RBFKernel(), x=x, y=y, y_std=None, hp_samples=500, observation_noise=False)
        if x is not None:
            gp.fit()
        x_star, y_star = acqf.optimize(gp, n=1000, method="Halton")
        y_exp = experiment(x_star)
    
        mu, sd = gp.predict(x_grid, fast=True)
        mu = mu.reshape(ppd, ppd).T
        sd = sd.reshape(ppd, ppd).T
        
        fig, axs = plt.subplots(1, 3, figsize=(6, 2), sharex=True)
        ax = axs[0]
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(mu, **plot_kwargs)
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Experiment")
    
        ax = axs[1]
        v = acqf(gp, x_grid)
        v = v.reshape(ppd, ppd).T
        if x is not None:
            ax.scatter(x[:, 0], x[:, 1])
        ax.imshow(v, **plot_kwargs)
        ax.scatter(x_star[:, 0], x_star[:, 1], marker="x", zorder=3)
        if iteration == 0:
            ax.set_title("Acquisition Function")
    
        ax = axs[2]
        ax.imshow(y_grid.reshape(ppd, ppd).T, **plot_kwargs)
        if iteration == 0:
            ax.set_title("Ground Truth")
        
        plt.show()
    
        if x is not None:
            x = np.append(x, x_star, axis=0)
            y = np.append(y, y_exp)
        else:
            x = np.atleast_2d(x_star).copy()
            y = np.atleast_1d(y_exp).copy()
